In [84]:
import pandas as pd
import numpy as np
from pprint import pprint
import json


In [2]:
data = pd.read_json('../assets/data/genre_network/top100-filtered.json')
data.head()

ValueError: arrays must all be same length

In [31]:
with open('../assets/data/genre_network/top100-filtered.json') as f:
    data = json.load(f)    

In [32]:
data.keys()

dict_keys(['directed', 'multigraph', 'graph', 'nodes', 'links'])

In [37]:
nodes = pd.DataFrame(data['nodes'])
nodes.isna().sum()

degree               0
katz_centrality      0
type                 0
artists            101
explicit           101
name               101
popularity         101
preview_url        101
album_type         101
album_id           101
album_name         101
release_date       101
genres             101
id                   1
dtype: int64

In [54]:
print(nodes.shape)
nodes.describe()

(7039, 14)


,degree,katz_centrality,popularity
count,7039.000000,7039.000000,6939.000000
mean,4.415542,0.001360,46.023923
std,20.444970,0.011814,17.574329
min,1.000000,-0.046524,0.000000
25%,1.000000,-0.000364,36.000000
50%,2.000000,-0.000028,48.000000
75%,3.000000,0.000424,58.000000
max,454.000000,0.571120,95.000000


In [88]:
# reduce data size to top 2000 track nodes + all genre nodes based on degree value
max_non_genre_node = nodes.loc[nodes.genres.isna()==False, :].nlargest(2000, columns='degree').degree.max()
node_ids = nodes.loc[nodes.genres.isna()==False, :].nlargest(2000, columns='degree').id.values
genre_node_ids = nodes.loc[(nodes.id.isna()==False) & (nodes.degree > max_non_genre_node), :].sort_values(by='degree', ascending=False).id.values

node_ids = np.concatenate([genre_node_ids, node_ids])

nodes = nodes.loc[nodes.id.isin(node_ids), :].reset_index(drop=True)
nodes


,degree,katz_centrality,type,artists,explicit,name,popularity,preview_url,album_type,album_id,album_name,release_date,genres,id
0,108,0.004365,genre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adult Standards
1,4,0.000418,track,[{'external_urls': {'spotify': 'https://open.s...,false,Down Town,52.0,https://p.scdn.co/mp3-preview/46a99678335c3054...,compilation,7emxnyckp3TBQ96yWi94kH,The Very Best Of Petula Clark,2015-07-17,"Adult Standards,Folk,Folk Rock,Mellow Gold",2IzUfd8Dv5wfvLoe8yfpNA
2,4,-0.001223,track,[{'external_urls': {'spotify': 'https://open.s...,false,Yesterday Once More,66.0,null,album,0A7WaXRUZ2fkHtUYfeoF73,Now & Then (Reissue),1973-05-01,"Adult Standards,Folk,Mellow Gold,Soft Rock",3wM6RTAnF7IQpMFd7b9ZcL
3,3,0.000636,track,[{'external_urls': {'spotify': 'https://open.s...,false,Try to Remember,46.0,https://p.scdn.co/mp3-preview/c6ad3c65bf312f96...,album,1LSxK62PrVWhpAK30scdYi,Try to Remember,1965,"Adult Standards,Folk,Folk Rock",6O5bX4w28pi72vJOTlEBTJ
4,5,0.000007,track,[{'external_urls': {'spotify': 'https://open.s...,false,Son of a Preacher Man,61.0,https://p.scdn.co/mp3-preview/50c7d07d328bef41...,album,6SNMenG3NjZyqgI5p4Hcmg,Dusty In Memphis [Deluxe Edition],1969-03-31,"Adult Standards,Folk,Folk Rock,Mellow Gold,Rock",7odHgoLFi3GQ90E9PeraI3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,73,-0.001273,genre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK Hip Hop
2096,128,-0.000169,genre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Underground Hip Hop
2097,162,-0.010448,genre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Urban Contemporary
2098,149,0.011634,genre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vapor Soul


In [33]:
links = pd.DataFrame(data['links'])
links

,source,target
0,24AIahNHzBxm9S12peXbnG,Adult Standards
1,24AIahNHzBxm9S12peXbnG,Mellow Gold
2,Adult Standards,4uxsv9PjV3Yeyn51RdWvGJ
3,Adult Standards,6crfO56bDm0RjpctUuGs5X
4,Adult Standards,55Z0KHofKqhOMIOygAVXxm
...,...,...
15538,Vapor Trap,69mR7YAMZmnV09SZGfudwT
15539,Vapor Trap,3rh664abW7EoAnROG5DF8n
15540,Vapor Trap,1PEEjg3x1ozQAj0xuDqY4R
15541,Vapor Trap,6EoSgWoIbQWWFGviinhQrt


In [36]:
links.dropna(inplace=True)

In [91]:
# remove links with a source or target that is not in the reduced dataset
links = links.loc[(links.source.isin(node_ids)) & (links.target.isin(node_ids)), :].reset_index(drop=True)
links

,source,target
0,Adult Standards,2IzUfd8Dv5wfvLoe8yfpNA
1,Adult Standards,3wM6RTAnF7IQpMFd7b9ZcL
2,Adult Standards,6O5bX4w28pi72vJOTlEBTJ
3,Adult Standards,7odHgoLFi3GQ90E9PeraI3
4,Adult Standards,2DnJjbjNTV9Nd5NOa1KGba
...,...,...
9042,1g8nfOOpggE0iyhomrk9fh,Trap
9043,1g8nfOOpggE0iyhomrk9fh,Vapor Trap
9044,1dhO5lyN2Otv4qR0xWQXH7,Trap
9045,1dhO5lyN2Otv4qR0xWQXH7,Underground Hip Hop


In [92]:
data = {}

data['nodes'] = json.loads(nodes.to_json(orient='records'))
data

{'nodes': [{'degree': 108,
   'katz_centrality': 0.0043645693,
   'type': 'genre',
   'artists': None,
   'explicit': None,
   'name': None,
   'popularity': None,
   'preview_url': None,
   'album_type': None,
   'album_id': None,
   'album_name': None,
   'release_date': None,
   'genres': None,
   'id': 'Adult Standards'},
  {'degree': 4,
   'katz_centrality': 0.0004179752,
   'type': 'track',
   'artists': "[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6nKqt1nbSBEq3iUXD1Xgz8'}, 'href': 'https://api.spotify.com/v1/artists/6nKqt1nbSBEq3iUXD1Xgz8', 'id': '6nKqt1nbSBEq3iUXD1Xgz8', 'name': 'Petula Clark', 'type': 'artist', 'uri': 'spotify:artist:6nKqt1nbSBEq3iUXD1Xgz8'}]",
   'explicit': 'false',
   'name': 'Down Town',
   'popularity': 52.0,
   'preview_url': 'https://p.scdn.co/mp3-preview/46a99678335c3054551a538f3f0204097082c644?cid=8abec43ed8f848bebb38ce429128d365',
   'album_type': 'compilation',
   'album_id': '7emxnyckp3TBQ96yWi94kH',
   'album_name': 'The Very B

In [93]:
data['links'] = json.loads(links.to_json(orient='records'))
data['links']

[{'source': 'Adult Standards', 'target': '2IzUfd8Dv5wfvLoe8yfpNA'},
 {'source': 'Adult Standards', 'target': '3wM6RTAnF7IQpMFd7b9ZcL'},
 {'source': 'Adult Standards', 'target': '6O5bX4w28pi72vJOTlEBTJ'},
 {'source': 'Adult Standards', 'target': '7odHgoLFi3GQ90E9PeraI3'},
 {'source': 'Adult Standards', 'target': '2DnJjbjNTV9Nd5NOa1KGba'},
 {'source': 'Adult Standards', 'target': '0HZlND4giwzgolBpaNIRGV'},
 {'source': 'Adult Standards', 'target': '2qhASBzpbFhPRtrnZ5lLnz'},
 {'source': 'Adult Standards', 'target': '71CXzHYYOyNqgtVFpNdeCS'},
 {'source': 'Adult Standards', 'target': '48i055G1OT5KxGGftwFxWy'},
 {'source': 'Adult Standards', 'target': '0Vth3l4XAo8hNYmxjzhr2y'},
 {'source': 'Adult Standards', 'target': '3l2hbXdvmaH3tUd3qWLKgm'},
 {'source': 'Adult Standards', 'target': '3Jy0dWV5AnnfyTvHK6UWof'},
 {'source': 'Adult Standards', 'target': '4tS8xQfyPGc7f4lJ1JP6BX'},
 {'source': 'Adult Standards', 'target': '2QDnRMFfnes9DUWc6HgixI'},
 {'source': 'Adult Standards', 'target': '5nNpVC

In [104]:
data['genres'] = json.dumps(genre_node_ids.tolist())
data.keys()

dict_keys(['nodes', 'links', 'genres'])

In [105]:
with open('../assets/data/genre_network/top100-filtered-reduced.json', 'w') as f:
    json.dump(data, f)    

In [47]:
links.loc[links.source=='24AIahNHzBxm9S12peXbnG', :]

,source,target
0,24AIahNHzBxm9S12peXbnG,Adult Standards
1,24AIahNHzBxm9S12peXbnG,Mellow Gold


In [48]:
nodes.loc[nodes.id=='24AIahNHzBxm9S12peXbnG', :]

,degree,katz_centrality,type,artists,explicit,name,popularity,preview_url,album_type,album_id,album_name,release_date,genres,id
0,2,0.000205,track,[{'external_urls': {'spotify': 'https://open.s...,false,"Moon River (From ""Breakfast at Tiffany's"")",65.0,https://p.scdn.co/mp3-preview/0123a76407f29dfc...,album,1RLfn8e4cQvC0WsYJM6rh2,Moon River And Other Great Movie Themes,1962,"Adult Standards,Mellow Gold",24AIahNHzBxm9S12peXbnG


In [102]:
data.keys()

dict_keys(['nodes', 'links'])

In [50]:
list(filter(lambda x: x['source'] == '24AIahNHzBxm9S12peXbnG',  data['links']))

[{'source': '24AIahNHzBxm9S12peXbnG', 'target': 'Adult Standards'},
 {'source': '24AIahNHzBxm9S12peXbnG', 'target': 'Mellow Gold'}]

In [51]:
nodes.isna().sum()

degree               0
katz_centrality      0
type                 0
artists            100
explicit           100
name               100
popularity         100
preview_url        100
album_type         100
album_id           100
album_name         100
release_date       100
genres             100
id                   0
dtype: int64

In [101]:
with open('../assets/data/genre_network/genres.json', 'w') as f:
    json.dump(genre_node_ids.tolist(), f)